In [1]:
import shap
import sklearn
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# 加载sklearn内置数据并查看其内容
breast_cancer = sklearn.datasets.load_breast_cancer()
breast_cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [3]:
# 构建建模所需数据集
breast_cancer_df = pd.DataFrame(dict(zip(breast_cancer.feature_names,breast_cancer.data.transpose(1,0))))
breast_cancer_df['type'] = list(breast_cancer.target)

In [4]:
# 观察数据集
breast_cancer_df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,type
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [5]:
# 导入pycaret聚类方法
from pycaret.clustering import *
# 初始环境创建，迭代次数10，指定归一化方法为zscore，固定随机种子
# 指定采用PCA（线性）降维
cluster_breast_cancer =  setup(data = breast_cancer_df,
                               iterative_imputation_iters =10,normalize=True,
                               normalize_method = 'zscore',session_id=666,
                               pca = True,pca_method='linear')


,Description,Value
0,session_id,666
1,Original Data,"(569, 31)"
2,Missing Values,False
3,Numeric Features,30
4,Categorical Features,1
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(569, 18)"
9,CPU Jobs,-1


In [6]:
# 构建kmeans模型，指定类别为2类（与真实情况匹配）
# 模型自动根据ground_truth列计算各项指标
kmeans = create_model('kmeans',num_clusters=2,ground_truth='type')

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.3543,277.9341,1.276,0.6024,0.7178,0.6333


In [14]:

# 绘制聚类可视化结果 
plot_model(kmeans, plot = 'cluster',scale =1 )

In [15]:
# 应用模型进行预测
k_df = assign_model(kmeans)

In [16]:
#计算聚类结果的正确性（）
cluster_result = [ ]
for i in k_df.Cluster:
    if i=='Cluster 1':
        cluster_result.append(1)
    else:
        cluster_result.append(0)

acc_cluster = np.array(cluster_result)*np.array(k_df.type)
len(acc_cluster[acc_cluster==0])/len(acc_cluster)

0.9876977152899824